In [1]:
# variables with suffix '_n' denote that nudged data are being used
# variables with prefix 'd' denote derivative

# dependencies
using MovingWeightedLeastSquares
using Plots
using Cubature
using Interpolations
using Base.Profile

# initialization of PGFPlots plotting backend
pgfplots();

In [922]:
# initialize the dataset in this Jupyter notebook cell

# from, to, step
from = -50;
to = 50;
step = .2;
# a random number in interval (-ng/2, ng/2) will be added to each x in xs
ng = .2;
# function to be approximated
f = x -> sin(2x)/2 + 10;
df = x -> cos(2x)
# weight function
w = (d, e) -> exp(-d^2/step^2);
# degree of used polynomial for approximation
deg = 2;
# eps of the weight function
eps = .6;
# attempts for time measurement
time_tries = 10;

In [924]:
rg = from:step:to
xs = collect(rg);
# comment the line below, if repeated evaluation with the same nudges is needed
rnd_n = ng * rand(size(xs)) - ng/2;
xs_n = xs .+ rnd_n;
fs = [f(x) for x in xs];
fs_n = [f(x) for x in xs_n];
dfs = [df(x) for x in xs];
dfs_n = [df(x) for x in xs_n];

# creation of approximation objects
nv = mwlsNaive(xs, fs, eps, w, maxDegree = deg);
nv_n = mwlsNaive(xs_n, fs_n, eps, w, maxDegree = deg);
kd = mwlsKd(xs, fs, eps, w, maxDegree = deg);
kd_n = mwlsKd(xs_n, fs_n, eps, w, maxDegree = deg);
cll = mwlsCll(xs, fs, eps, w, maxDegree = deg);
cll_n = mwlsCll(xs_n, fs_n, eps, w, maxDegree = deg);
xs

501-element Array{Float64,1}:
 -50.0
 -49.8
 -49.6
 -49.4
 -49.2
 -49.0
 -48.8
 -48.6
 -48.4
 -48.2
 -48.0
 -47.8
 -47.6
   ⋮  
  47.8
  48.0
  48.2
  48.4
  48.6
  48.8
  49.0
  49.2
  49.4
  49.6
  49.8
  50.0

In [925]:
dnvs = mass_diff(nv, xs);

In [926]:
@time for i in 1:time_tries
    mass_diff(nv, xs)
end

 19.395030 seconds (89.05 M allocations: 3.046 GiB, 8.35% gc time)


In [927]:
dkds = mass_diff(kd, xs);

In [944]:
@time for i in 1:time_tries
    mass_diff(kd, xs);
end

  2.873261 seconds (11.63 M allocations: 340.878 MiB, 6.53% gc time)


In [929]:
dclls = mass_diff(cll, xs);

In [930]:
@time for i in 1:time_tries
    mass_diff(cll, xs);
end

  5.405187 seconds (18.78 M allocations: 1020.478 MiB, 7.47% gc time)


In [931]:
dnvs_n = mass_diff(nv_n, xs_n);

In [932]:
@time for i in 1:time_tries
    mass_diff(nv_n, xs_n)
end

 19.060754 seconds (87.11 M allocations: 2.995 GiB, 8.58% gc time)


In [933]:
dkds_n = mass_diff(kd_n, xs_n);

In [934]:
@time for i in 1:time_tries
    mass_diff(kd_n, xs_n);
end

  3.002485 seconds (11.92 M allocations: 348.607 MiB, 6.29% gc time)


In [935]:
dclls_n = mass_diff(cll_n, xs_n);

In [936]:
@time for i in 1:time_tries
    mass_diff(cll_n, xs_n);
end

  4.519199 seconds (16.83 M allocations: 967.509 MiB, 8.02% gc time)


In [937]:
# init of the interpolation object from Interpolations.jl
itp = scale(interpolate(fs, BSpline(Quadratic(Free())), OnGrid()), rg);

In [938]:
# init of the interpolation object using nudged outputs
itp_n = scale(interpolate(fs_n, BSpline(Quadratic(Free())), OnGrid()), rg);

In [939]:
ditps = interpolations_mass_diff(itp, xs);

In [940]:
@time for i in 1:time_tries
    interpolations_mass_diff(itp, xs)
end

  0.000689 seconds (10.06 k allocations: 589.406 KiB)


In [941]:
ditps_n = interpolations_mass_diff(itp_n, xs_n);

In [942]:
@time for i in 1:time_tries
    interpolations_mass_diff(itp_n, xs_n)
end

  0.000460 seconds (10.06 k allocations: 589.406 KiB)
